In [28]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\Training\\Raw\\Cluster_8.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,12ema,26ema,ATR,Apr,Aug,Dec,Feb,Fifth,First,Fourth,...,lower_band,ma21,ma7,open,origin_close,return,total,ts,upper_band,vol
0,20.725903,20.865749,0.758496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.414137,20.960779,20.628722,20.554629,21.80,-0.40,256934048.0,2004-03-16,21.507422,1.452482e+07
1,20.668146,20.822031,0.754619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.334061,20.916606,20.562205,20.484267,21.20,-0.60,225470767.0,2004-03-17,21.499151,3.884793e+06
2,20.607916,20.776216,0.751195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.252282,20.869062,20.493587,20.411374,22.00,0.80,268451257.0,2004-03-18,21.485843,1.095667e+07
3,20.545111,20.728229,0.748228,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,20.168364,20.818313,20.422861,20.335698,22.30,0.30,179133742.0,2004-03-19,21.468262,1.244231e+07
4,20.479874,20.678124,0.745778,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,20.082638,20.764626,20.350363,20.259512,20.80,-1.50,7977421.0,2004-03-22,21.446613,1.097741e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542090,20.285957,20.342582,0.719329,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,19.960548,20.454071,20.219568,20.197557,19.30,-0.20,10570940.0,2019-09-17,20.947593,2.759585e+06
542091,20.255189,20.323573,0.706321,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,19.964582,20.411484,20.184386,20.164403,19.15,-0.15,8499084.0,2019-09-18,20.858386,2.248338e+06
542092,20.222508,20.302772,0.692764,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,19.969071,20.367657,20.147611,20.117531,18.80,-0.35,18002738.0,2019-09-19,20.766244,2.669097e+06
542093,20.185653,20.279082,0.678646,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,19.958678,20.323679,20.107630,20.053132,19.15,0.35,10280286.0,2019-09-20,20.688680,2.191557e+06


In [4]:
train_df, val_df, test_df = func.TrainTestSplit(data, date(2016,9,1), date(2017,9,1))

D:\庫存健診開發\code\TrainingPrep\Training_function.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_df = data[data['ts'].dt.date < ValDate][data['ts'].dt.date >= TrainDate]


In [6]:
df_param = pd.concat([train_df, val_df], axis=0)
df_param

,12ema,26ema,ATR,Apr,Aug,Dec,Feb,Fifth,First,Fourth,...,lower_band,ma21,ma7,open,origin_close,return,total,ts,upper_band,vol
0,20.725903,20.865749,0.758496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.414137,20.960779,20.628722,20.554629,21.80,-0.40,256934048.0,2004-03-16,21.507422,1.452482e+07
1,20.668146,20.822031,0.754619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.334061,20.916606,20.562205,20.484267,21.20,-0.60,225470767.0,2004-03-17,21.499151,3.884793e+06
2,20.607916,20.776216,0.751195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.252282,20.869062,20.493587,20.411374,22.00,0.80,268451257.0,2004-03-18,21.485843,1.095667e+07
3,20.545111,20.728229,0.748228,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,20.168364,20.818313,20.422861,20.335698,22.30,0.30,179133742.0,2004-03-19,21.468262,1.244231e+07
4,20.479874,20.678124,0.745778,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,20.082638,20.764626,20.350363,20.259512,20.80,-1.50,7977421.0,2004-03-22,21.446613,1.097741e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541585,13.764485,13.719204,0.312829,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,13.525866,13.694696,13.797859,13.936613,13.35,0.00,409988.0,2017-08-25,13.863526,-1.189097e+05
541586,13.782611,13.731286,0.316773,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,13.523698,13.707123,13.818148,13.957767,13.60,0.25,1787926.0,2017-08-28,13.890547,3.043028e+04
541587,13.800253,13.743582,0.320668,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,13.524703,13.720283,13.838056,13.974009,14.05,0.45,12537192.0,2017-08-29,13.915863,5.691239e+05
541588,13.817207,13.755943,0.324491,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,13.529364,13.734216,13.857459,13.988354,14.35,0.30,8826324.0,2017-08-30,13.939068,5.780179e+05


In [8]:
feature_list = ['12ema', '26ema', 'ATR',
                'MACD', 'RSI_15', 'SO%d3',
                'SO%k5', 'TR', 'VWAP', 'WeekNum',
                'close', 'high', 'low', 'lower_band', 'ma21', 'ma7', 'open',
                'upper_band', 'origin_close']

param = func.GetScalerParam(df_param, feature_list)
param

,max,min,std,mean
12ema,1212.722593,9.892427e-01,60.394205,35.559816
26ema,1183.298913,9.972108e-01,60.374872,35.631214
ATR,87.129862,2.129330e-02,2.304856,1.086195
MACD,60.124989,-7.026317e+01,2.107555,-0.071398
RSI_15,1.000000,0.000000e+00,0.426508,0.465568
SO%d3,1.220897,-4.427066e+00,0.172435,0.440907
SO%k5,1.644025,-8.406533e+00,0.177335,0.440930
TR,204.204268,-2.215757e-02,2.442811,1.082218
VWAP,574943.903436,-2.120541e+06,3885.006163,30.012471
WeekNum,1.000000,0.000000e+00,0.307353,0.534091


In [10]:
train_df, val_df, test_df = func.Scaler(train_df, feature_list, param), func.Scaler(val_df, feature_list, param), func.Scaler(test_df, feature_list, param)

In [31]:
df_list = [group[1] for group in train_df.groupby(train_df['StockNo'])]

X_train, Y_train = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset, lookback=20, forward=5, problem='classification'), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_train.append(xx.tolist())
            for yy in x[1]:
                Y_train.append(yy.tolist())


MemoryError: 

In [29]:
df_list = [group[1] for group in val_df.groupby(val_df['StockNo'])]

X_val, Y_val = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset, lookback=20, forward=5, problem='classification'), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_val.append(xx.tolist())
            for yy in x[1]:
                Y_val.append(yy.tolist())

np.array(X_val).shape

(36300, 20, 26)

In [36]:
df_list = [group[1] for group in test_df.groupby(test_df['StockNo'])]

X_test, Y_test = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset, lookback=20, forward=5, problem='classification'), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_test.append(xx.tolist())
            for yy in x[1]:
                Y_test.append(yy.tolist())

np.array(X_test).shape

(79200, 20, 26)

In [37]:
(len(X_train) == len(Y_train)) and (len(X_val) == len(Y_val)) and (len(X_test) == len(Y_test))

True

In [38]:
save_list = [X_train, Y_train, X_val, Y_val, X_test, Y_test]

In [40]:
cluster = 8
import pickle
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\Cluster_{cluster}\\Cluster_{cluster}_classification_minmax0_Weekly_withoutMonth', 'wb') as fp:
    pickle.dump(save_list, fp)